In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        if row['inc_angle'] == 'na':
            ang = -1
        else:
            ang = float(row['inc_angle'])
        
        band_3 = (band_1 - band_2)*ang/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input,BatchNormalization
from keras.models import Model

def create_vgg_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='linear', padding='same', name='block1_conv2')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='linear', padding='same', name='block2_conv2')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='linear', padding='same', name='block3_conv3')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='linear', padding='same', name='block4_conv3')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='linear', padding='same', name='block5_conv3')(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = Flatten()(x)
    x = Dense(1024,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [5]:
tmp_m = create_vgg_model()
tmp_m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 75, 75, 64)        0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
__________

In [6]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator


def lr_f(epoch):
    if epoch<10:
        return 0.0005
    elif epoch<30:
        return 0.00005
    else:
        return 0.00001

def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 32
        steps_train = len(curr_y)//bat_size
        
        
        model = create_vgg_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=80, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/80
Epoch 00001: val_loss improved from inf to 4.41854, saving model to best_m.h5
 - 15s - loss: 1.0926 - acc: 0.5752 - val_loss: 4.4185 - val_acc: 0.5402
Epoch 2/80
Epoch 00002: val_loss did not improve
 - 11s - loss: 0.5043 - acc: 0.7843 - val_loss: 4.5725 - val_acc: 0.5402
Epoch 3/80
Epoch 00003: val_loss did not improve
 - 11s - loss: 0.3942 - acc: 0.8169 - val_loss: 4.4240 - val_acc: 0.5402
Epoch 4/80
Epoch 00004: val_loss improved from 4.41854 to 3.85542, saving model to best_m.h5
 - 11s - loss: 0.3873 - acc: 0.8510 - val_loss: 3.8554 - val_acc: 0.5402
Epoch 5/80
Epoch 00005: val_loss improved from 3.85542 to 2.48364, saving model to best_m.h5
 - 11s - loss: 0.3889 - acc: 0.8419 - val_loss: 2.4836 - val_acc: 0.5402
Epoch 6/80
Epoch 00006: val_loss did not improve
 - 11s - loss: 0.3336 - acc: 0.8728 - val_loss: 3.5132 - val_acc: 0.5402
Epoch 7/80
Epoch 00007: val_loss did not improve
 - 11s - loss: 0.3171 - acc: 0.8732 - val_loss: 3.4381 - val_acc: 0.5402
Epoch 8/80
Epoch 0

Epoch 63/80
Epoch 00063: val_loss did not improve
 - 11s - loss: 0.2071 - acc: 0.9221 - val_loss: 0.2352 - val_acc: 0.8897
Epoch 64/80
Epoch 00064: val_loss did not improve
 - 11s - loss: 0.1676 - acc: 0.9413 - val_loss: 0.2417 - val_acc: 0.8822
Epoch 65/80
Epoch 00065: val_loss did not improve
 - 11s - loss: 0.1921 - acc: 0.9242 - val_loss: 0.2281 - val_acc: 0.8935
Epoch 66/80
Epoch 00066: val_loss did not improve
 - 11s - loss: 0.1792 - acc: 0.9356 - val_loss: 0.2513 - val_acc: 0.8822
Epoch 67/80
Epoch 00067: val_loss did not improve
 - 11s - loss: 0.1717 - acc: 0.9352 - val_loss: 0.2234 - val_acc: 0.8916
Epoch 68/80
Epoch 00068: val_loss did not improve
 - 11s - loss: 0.1736 - acc: 0.9413 - val_loss: 0.2220 - val_acc: 0.8841
Epoch 69/80
Epoch 00069: val_loss did not improve
 - 11s - loss: 0.1878 - acc: 0.9177 - val_loss: 0.2558 - val_acc: 0.8748
Epoch 70/80
Epoch 00070: val_loss did not improve
 - 11s - loss: 0.1663 - acc: 0.9361 - val_loss: 0.2234 - val_acc: 0.8935
Epoch 71/80
Epoc

Epoch 45/80
Epoch 00045: val_loss improved from 0.22022 to 0.21840, saving model to best_m.h5
 - 12s - loss: 0.2122 - acc: 0.9096 - val_loss: 0.2184 - val_acc: 0.9196
Epoch 46/80
Epoch 00046: val_loss improved from 0.21840 to 0.21489, saving model to best_m.h5
 - 11s - loss: 0.1985 - acc: 0.9185 - val_loss: 0.2149 - val_acc: 0.9234
Epoch 47/80
Epoch 00047: val_loss did not improve
 - 11s - loss: 0.2036 - acc: 0.9124 - val_loss: 0.2183 - val_acc: 0.9215
Epoch 48/80
Epoch 00048: val_loss did not improve
 - 11s - loss: 0.1966 - acc: 0.9176 - val_loss: 0.2245 - val_acc: 0.9178
Epoch 49/80
Epoch 00049: val_loss did not improve
 - 11s - loss: 0.1918 - acc: 0.9310 - val_loss: 0.2193 - val_acc: 0.9178
Epoch 50/80
Epoch 00050: val_loss did not improve
 - 11s - loss: 0.1996 - acc: 0.9214 - val_loss: 0.2384 - val_acc: 0.9065
Epoch 51/80
Epoch 00051: val_loss improved from 0.21489 to 0.21096, saving model to best_m.h5
 - 11s - loss: 0.2177 - acc: 0.9016 - val_loss: 0.2110 - val_acc: 0.9196
Epoch 5

Epoch 27/80
Epoch 00027: val_loss improved from 0.23801 to 0.23364, saving model to best_m.h5
 - 11s - loss: 0.2929 - acc: 0.8844 - val_loss: 0.2336 - val_acc: 0.8933
Epoch 28/80
Epoch 00028: val_loss did not improve
 - 11s - loss: 0.3208 - acc: 0.8650 - val_loss: 0.2974 - val_acc: 0.8820
Epoch 29/80
Epoch 00029: val_loss improved from 0.23364 to 0.22079, saving model to best_m.h5
 - 11s - loss: 0.2696 - acc: 0.8889 - val_loss: 0.2208 - val_acc: 0.9082
Epoch 30/80
Epoch 00030: val_loss did not improve
 - 11s - loss: 0.2660 - acc: 0.8958 - val_loss: 0.2446 - val_acc: 0.9007
Epoch 31/80
Epoch 00031: val_loss did not improve
 - 11s - loss: 0.2605 - acc: 0.8861 - val_loss: 0.2490 - val_acc: 0.8989
Epoch 32/80
Epoch 00032: val_loss did not improve
 - 11s - loss: 0.2558 - acc: 0.8908 - val_loss: 0.2503 - val_acc: 0.8951
Epoch 33/80
Epoch 00033: val_loss did not improve
 - 11s - loss: 0.2549 - acc: 0.8913 - val_loss: 0.2209 - val_acc: 0.9082
Epoch 34/80
Epoch 00034: val_loss improved from 0.2

In [7]:
import pickle
with open('../2nd_features/vgg_aug1_new_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../2nd_features/vgg_aug1_new_sub.csv', index=False)

# pre 2370
# new 2088
# new 2033

0.200412109699
         id  is_iceberg
0  5941774d    0.052610
1  4023181e    0.968007
2  b20200e4    0.332817
3  e7f018bb    0.989567
4  4371c8c3    0.221182
